In [13]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

# Create list of tours

## Highest-grossing tours 
https://en.wikipedia.org/wiki/List_of_highest-grossing_concert_tours

In [14]:
url = "https://en.wikipedia.org/wiki/List_of_highest-grossing_concert_tours"

In [15]:
response = requests.get(
	url=url,
)
soup = BeautifulSoup(response.content, 'html.parser')

captions = [
    "Top 20 highest-grossing tours of all time",
    "Top 10 highest-grossing tours of the 1980s",
    "Top 10 highest-grossing tours of the 1990s",
    "Top 10 highest-grossing tours of the 2000s",
    "Top 10 highest-grossing tours of the 2010s",
    "Top 10 highest-grossing tours of the 2020s"
]

In [16]:
URL = "https://en.wikipedia.org/wiki/List_of_highest-grossing_concert_tours"
def create_tour_list(captions, url):
    df = pd.DataFrame()
    for caption in captions:
        df_tmp = pd.read_html(url, 
                          match=caption)[0]
        
        tour_links = pd.read_html(url, 
                  match=caption,
                  extract_links="all")[0][("Tour title", None)]
        
        tour_links = pd.DataFrame(tour_links.tolist(), columns=["Tour title", "Tour link"])
        df_tmp = df_tmp.merge(tour_links, on="Tour title")

        df = pd.concat([df, df_tmp]).drop_duplicates('Tour title').reset_index(drop=True)
        print(df.shape)
    
    return df

df = create_tour_list(captions, url)

(20, 11)
(30, 12)
(40, 12)
(45, 12)
(45, 12)
(49, 12)


In [17]:
df = df.rename(columns={df.columns[3]: 'Adjusted gross (in 2022 dollars)',
                   df.columns[-1]: 'Adjusted gross (in 2022 dollar)',
                   df.columns[2]: 'Actual gross'})

In [18]:
df.loc[df['Adjusted gross (in 2022 dollars)'].isna(), 'Adjusted gross (in 2022 dollars)'] = df['Adjusted gross (in 2022 dollar)']

In [19]:
df = df[["Actual gross", "Adjusted gross (in 2022 dollars)", "Artist", "Tour title", "Tour link", "Year(s)", "Shows"]]

In [20]:
df.head()

,Actual gross,Adjusted gross (in 2022 dollars),Artist,Tour title,Tour link,Year(s),Shows
0,"$939,100,000","$939,100,000",Elton John,Farewell Yellow Brick Road,/wiki/Farewell_Yellow_Brick_Road,2018–2023,330
1,"$780,000,000","$780,000,000",Taylor Swift,The Eras Tour †,/wiki/The_Eras_Tour,2023,56
2,"$776,200,000","$888,442,379",Ed Sheeran,÷ Tour,/wiki/%C3%B7_Tour,2017–2019,255
3,"$736,421,586","$958,001,690",U2,U2 360° Tour,/wiki/U2_360%C2%B0_Tour,2009–2011,110
4,"$667,726,905","$667,726,905",Coldplay,Music of the Spheres World Tour †,/wiki/Music_of_the_Spheres_World_Tour,2022–2023,114


## List of most-attended concert tours
https://en.wikipedia.org/wiki/List_of_most-attended_concert_tours

In [21]:
captions = [
    "Tours attended by 5 million people or more",
    "Tours attended by 3.5 to 4.9 million people"
]
URL = "https://en.wikipedia.org/wiki/List_of_most-attended_concert_tours"

df_att = create_tour_list(captions, URL)

(14, 7)
(31, 7)


In [22]:
df_att = df_att[["Year(s)", "Tour title", "Tour link", "Artist", "Shows", "Tickets sold"]]
df = pd.concat([df, df_att]).drop_duplicates('Tour link').reset_index(drop=True)

In [11]:
df.head(5)

,Actual gross,Adjusted gross (in 2022 dollars),Artist,Tour title,Tour link,Year(s),Shows,Tickets sold
0,"$939,100,000","$939,100,000",Elton John,Farewell Yellow Brick Road,/wiki/Farewell_Yellow_Brick_Road,2018–2023,330,NaN
1,"$780,000,000","$780,000,000",Taylor Swift,The Eras Tour †,/wiki/The_Eras_Tour,2023,56,NaN
2,"$776,200,000","$888,442,379",Ed Sheeran,÷ Tour,/wiki/%C3%B7_Tour,2017–2019,255,NaN
3,"$736,421,586","$958,001,690",U2,U2 360° Tour,/wiki/U2_360%C2%B0_Tour,2009–2011,110,NaN
4,"$667,726,905","$667,726,905",Coldplay,Music of the Spheres World Tour †,/wiki/Music_of_the_Spheres_World_Tour,2022–2023,114,NaN


I remove:
- index 50 - duplicated tour
- index 35 - wrong wikipedia page, no info about tour

In [27]:
df = df.drop(index=[50, 35]).reset_index()

# Gather data without cancelled shows

In [30]:
BASE_URL = "https://en.wikipedia.org"
for index, row in df.iterrows():
    url = BASE_URL + row["Tour link"]
    response = requests.get(url=url)

    soup = BeautifulSoup(response.content, 'html.parser')
    # print(f"{row['Tour title']}")
    for t in soup.body.findAll("table"):
        # print("Cancelled shows" in str(t.find_previous_sibling()))

        if "Cancelled shows" in str(t.find_previous_sibling()):
            t.clear()


I'm gonna combine this piece of code with code that is parsing entire tables

In [31]:
BASE_URL = "https://en.wikipedia.org"
df_tours = pd.DataFrame()

for index, row in df.iterrows():
    url = BASE_URL + row["Tour link"]
    response = requests.get(url=url)

    soup = BeautifulSoup(response.content, 'html.parser')
    for t in soup.body.findAll("table"):
        if "Cancelled shows" in str(t.find_previous_sibling()):
            t.clear()

    tables = pd.read_html(str(soup))
    df_tour_tmp = pd.DataFrame()
    for t in tables:
        if "City" in t.columns and "Country" in t.columns:
            t.columns = t.columns.get_level_values(0)
            for i, _ in enumerate(t.columns):
                if t.columns[i].startswith("Date"):
                    t = t.rename(columns={t.columns[i] : "Date"})
            df_tour_tmp = pd.concat([df_tour_tmp, t[["Country", "City", "Venue", "Date"]]])
    df_tour_tmp = df_tour_tmp.drop(df_tour_tmp[df_tour_tmp["Date"] == df_tour_tmp["City"]].index)
    df_tour_tmp["Artist"] = row["Artist"]
    df_tour_tmp["Tour title"] = row["Tour title"]
    
    df_tours = pd.concat([df_tours, df_tour_tmp])
    # print(f"{row['Tour title']}")

In [34]:
df_tours = df_tours.reset_index(drop=True)

In [32]:
df_tours.shape

(7601, 6)

Almost hundred of shows were rejected - before skipping cancelled ones there was 7696 shows.

# Geolocation of cities

In [35]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my_user_agent_3")

Many venues are recurring, so it may be good idea to cache already geolocated places in dictionary object

In [36]:
cached_locations = {}

def geolocalize_cities(df_tours):
    for index, row in df_tours.iterrows():
        venue = f"{df_tours.iloc[index, 1]}, {df_tours.iloc[index, 0]}"
        if venue in cached_locations:
            df_tours.loc[index, "Latitude"] = cached_locations[venue][0]
            df_tours.loc[index, "Longtitude"] = cached_locations[venue][1]
        else:
            location = geolocator.geocode(venue, timeout=10)
            if not location:
                print(venue)
                continue
            df_tours.loc[index, "Latitude"] = location.latitude
            df_tours.loc[index, "Longtitude"] = location.longitude
            cached_locations[venue] = [location.latitude, location.longitude]


In [37]:
geolocalize_cities(df_tours)

Langraaf, Netherlands
Hanover, West Germany
Moscow, Soviet Union
Moscow, Soviet Union
Moscow, Soviet Union
Moscow, Soviet Union
Moscow, Soviet Union
Cologne, West Germany
Cologne, West Germany
Hockenheim, West Germany
Würzburg, West Germany
Hanover, West Germany
Hanover, West Germany
Cologne, West Germany
Cologne, West Germany
East Berlin, East Germany
East Berlin, East Germany
Prague, Czechoslovakia
Nürburgring, West Germany
Slane, Ireland, Republic of
Nuremberg, West Germany
Nuremberg, West Germany
Hanover, West Germany
Hanover, West Germany
Hanover, West Germany
Mursfreesboro, United States
Cologne, West Germany
Taipei City, Taiwan[21][22]
Kaohsiung, Taiwan[21][22]
Taipei City, Taiwan[21][22]


Now I know, that 30 places aren't geolocalised properly:
    - Some had additional characters attached to country name (Taiwan[21][22])
    - Some had country that is no longer existing (like Moscow, Soviet Union; Hanover - West Germany)
    - Some had misspelings (Langraaf -> Landgraaf; Mursfreesboro->Murfreesboro)

In [77]:
df_tours.loc[df_tours["Country"] == "West Germany", "Country"] = "Germany"
df_tours.loc[df_tours["Country"] == "Ireland, Republic of", "Country"] = "Ireland"
df_tours.loc[df_tours["Country"] == "Republic of Ireland", "Country"] = "Ireland"
df_tours.loc[df_tours["Country"] == "Czechoslovakia", "Country"] = "Czech Republic"
df_tours.loc[df_tours["Country"] == "Soviet Union", "Country"] = "Russia"
df_tours.loc[df_tours["Country"] == "East Germany", "Country"] = "Germany"
df_tours.loc[df_tours["Country"] == "Perú", "Country"] = "Peru"


df_tours.loc[df_tours["Country"] == "Taiwan[21][22]", "Country"] = "Taiwan"

df_tours.loc[df_tours["City"] == "Langraaf", "City"] = "Landgraaf"
df_tours.loc[df_tours["City"] == "Mursfreesboro", "City"] = "Murfreesboro"
df_tours.loc[df_tours["City"] == "East Berlin", "City"] = "Berlin"


In [39]:
geolocalize_cities(df_tours)

## Add continents data

In [54]:
import csv

country_to_continent = {}

with open('Continents_to_CountryNames.csv') as f:
    next(f)
    reader = csv.reader(f)
    for row in reader:
        country_to_continent[row[1]] = row[0]


In [91]:
country_to_continent["England"] = "Europe"
country_to_continent["Scotland"] = "Europe"
country_to_continent["Ireland"] = "Europe"
country_to_continent["Ireland"] = "Europe"
country_to_continent["Wales"] = "Europe"
country_to_continent["Northern Ireland"] = "Europe"
country_to_continent["Puerto Rico"] = "North America"
country_to_continent["Taiwan"] = "Asia"
country_to_continent["Hong Kong"] = "Asia"
country_to_continent["South Korea"] = "Asia"
country_to_continent["Russia"] = "Europe"
country_to_continent["Macau"] = "Russia"

In [59]:
country_to_continent["England"]

'Europe'

In [83]:
df_tours

,Country,City,Venue,Date,Artist,Tour title,Latitude,Longtitude
0,United States,Allentown,PPL Center,8 September 2018,Elton John,Farewell Yellow Brick Road,40.602206,-75.471279
1,United States,Philadelphia,Wells Fargo Center,11 September 2018,Elton John,Farewell Yellow Brick Road,39.952724,-75.163526
2,United States,Philadelphia,Wells Fargo Center,12 September 2018,Elton John,Farewell Yellow Brick Road,39.952724,-75.163526
3,United States,Buffalo,KeyBank Center,15 September 2018,Elton John,Farewell Yellow Brick Road,37.709767,-95.697482
4,United States,University Park,Bryce Jordan Center,16 September 2018,Elton John,Farewell Yellow Brick Road,41.440034,-87.683377
...,...,...,...,...,...,...,...,...
7596,United States,Charlotte,PNC Music Pavilion,27 September 2014,One Direction,Where We Are Tour,35.227209,-80.843083
7597,United States,Charlotte,PNC Music Pavilion,28 September 2014,One Direction,Where We Are Tour,35.227209,-80.843083
7598,United States,Atlanta,Georgia Dome,1 October 2014,One Direction,Where We Are Tour,33.748992,-84.390264
7599,United States,Tampa,Raymond James Stadium,3 October 2014,One Direction,Where We Are Tour,27.947760,-82.458444


In [88]:
df_tours[df_tours["Country"].isna()].index

Index([2460, 2461, 3015], dtype='int64')

In [89]:
df_tours[df_tours["Country"].isna()]
df_tours = df_tours.drop(index=df_tours[df_tours["Country"].isna()].index )

In [93]:
df_tours[df_tours["Country"] == "Oakland–Alameda County Coliseum Arena"]

,Country,City,Venue,Date,Artist,Tour title,Latitude,Longtitude
3871,Oakland–Alameda County Coliseum Arena,Oakland,NaN,24 October 1986,Genesis,Invisible Touch Tour,37.750273,-122.202932


In [95]:
df_tours.loc[df_tours["Country"] == "Oakland–Alameda County Coliseum Arena", "Venue"] = "Oakland–Alameda County Coliseum Arena"
df_tours.loc[df_tours["Country"] == "Oakland–Alameda County Coliseum Arena", "Country"] = "United States"

In [96]:
df_tours["Continent"] = df_tours["Country"].apply(lambda x : country_to_continent[x])

In [64]:
df_tours.to_csv("prepared_data.csv", encoding="utf-8")